In [1]:
# for part 4.
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.layers import Dropout, Input, BatchNormalization, Reshape
from keras.layers import LSTM, Bidirectional
from keras.optimizers import SGD

import numpy as np

import os
import csv
import time

Using TensorFlow backend.


In [2]:
# csv_files_train = "/home/danny/Repos/text_recognition/tf-crnn-master/data/train.csv"
# csv_files_eval = "/home/danny/Repos/text_recognition/tf-crnn-master//data/valid.csv"
# output_model_dir = "/home/danny/Repos/text_recognition/tf-crnn-master/estimator"
csv_files_train = "C:/Users/danny/Repos/text_recognition/tf-crnn-master/data/train.csv"
csv_files_eval = "C:/Users/danny/Repos/text_recognition/tf-crnn-master//data/valid.csv"
output_model_dir = "C:/Users/danny/Repos/text_recognition/tf-crnn-master/estimator"
n_epochs = 5
gpu = "" # help="GPU 0,1 or '' ", default=''

train_batch_size=64
eval_batch_size=64
learning_rate=1e-3  # 1e-3 recommended
learning_decay_rate=0.95
learning_decay_steps=5000
evaluate_every_epoch=5
save_interval=5e3
input_shape=(117, 1669, 1)
optimizer='adam'
digits_only=False
alphabet=" !\"#&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz|~"
alphabet_decoding='same'
alphabet_codes = list(range(len(alphabet)))
n_classes = len(alphabet)
csv_delimiter='\t'
keep_prob_dropout = 0.7

# Start making the model

In [3]:
model = Sequential()

## The deep cnn part

In [4]:
# conv1 - maxPool2x2
model.add(Conv2D(64, (3,3), activation="relu", input_shape=input_shape,
                 padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

# conv2 - maxPool2x2
model.add(Conv2D(128, (3,3), activation="relu", input_shape=input_shape,
                 padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

# conv3 - w/batch-norm
model.add(Conv2D(256, (3,3), input_shape=input_shape, padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))

# conv4 - maxPool 2x1
model.add(Conv2D(256, (3,3), activation="relu", input_shape=input_shape,
                 padding="same"))
model.add(MaxPooling2D(pool_size=(2, 1), padding="same"))

# conv5 - w/batch-norm
model.add(Conv2D(512, (3,3), input_shape=input_shape, padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))

# conv6 - maxPool 2x1
model.add(Conv2D(512, (3,3), activation="relu", input_shape=input_shape,
                 padding="same"))
model.add(MaxPooling2D(pool_size=(2, 1), padding="same"))

# conv 7 - w/batch-norm
model.add(Conv2D(512, (2,2), input_shape=input_shape, padding="valid"))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))

# reshape output
# from [batch, height, width, features]
# to [batch, width, height x features]
shp = model.layers[-1].output_shape
model.add(Reshape((shp[2], shp[1]*shp[3])))

## Bidirectional LSTM

In [5]:
n_hidden = [256, 256]
for nh in n_hidden:
    model.add(Bidirectional(LSTM(nh, return_sequences=True)))

model.add(Dropout(keep_prob_dropout))
model.add(Dense(n_classes))

In [ ]:
model.add(Activation("softmax"))

## Probably something else just a placeholder for now

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Flatten())

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop')

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

# higher verbose means more printing
model.fit(trX, trY, epochs=1000, verbose=0)

weights = model.layers[0].get_weights()
w_final = weights[0][0][0]
b_final = weights[1][0]
print('Linear regression model is trained to have weight w: %.2f, b: %.2f' % (w_final, b_final))

In [ ]:
model.save_weights("my_model.h5")

model.load_weights("my_model.h5")

In [ ]:
digit_input = Input(shape=(28, 28, 1))

x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)
out = Flatten()(x)

vision_model = Model(digit_input, out)

# this code won't run
img_input = Input(shape=input_shape)
# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
 
# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
 
# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
 
# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
 
# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
 
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(classes, activation='softmax', name='predictions')(x)